In [ ]:
# AI Prediction Stock GUI

import os
import tkinter as tk
from tkinter import ttk
import pandas as pd
import subprocess
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg, NavigationToolbar2Tk

# Create the main window
root = tk.Tk()
root.title("AI Prediction Stock")

# Define color gradient
def calculate_color_gradient(position, total_length):
    # Calculate the normalized position in the range [0, 1]
    normalized_position = position / total_length
    # Define the starting and ending colors (e.g., white to light blue)
    start_color = (255, 255, 255)  # White
    end_color = (173, 216, 230)     # Light blue
    # Interpolate between the starting and ending colors based on the normalized position
    r = int(start_color[0] * (1 - normalized_position) + end_color[0] * normalized_position)
    g = int(start_color[1] * (1 - normalized_position) + end_color[1] * normalized_position)
    b = int(start_color[2] * (1 - normalized_position) + end_color[2] * normalized_position)
    # Return the RGB color as a hexadecimal string
    return f'#{r:02x}{g:02x}{b:02x}'

# Function to load data and display DataFrame
def load_data_and_display():
    ticker = symbol_combobox.get()
    # Assuming load_data, N_STEPS, SCALE, SPLIT_BY_DATE, SHUFFLE, LOOKUP_STEP,
    # TEST_SIZE, and FEATURE_COLUMNS are defined elsewhere
    data = load_data(ticker, N_STEPS, scale=SCALE, split_by_date=SPLIT_BY_DATE,
                     shuffle=SHUFFLE, lookup_step=LOOKUP_STEP, test_size=0.3,
                     feature_columns=FEATURE_COLUMNS)
    ticker_data_filename = f"{ticker}_data.csv"  # Adjust filename as needed
    # data["df"].to_csv(ticker_data_filename)
    data["df"].to_csv(ticker_data_filename, index_label="date")
    df = pd.read_csv(ticker_data_filename)
    
    # Display DataFrame in a new window
    df_window = tk.Toplevel(root)
    df_window.title("Stock Data")
    df_frame = ttk.Frame(df_window)
    df_frame.pack(padx=10, pady=10)
    df_label = ttk.Label(df_frame, text="Stock Data:")
    df_label.grid(row=0, column=0, padx=5, pady=5)
    
    # Create a Text widget with a vertical scrollbar
    df_text = tk.Text(df_frame, height=30, width=100)
    df_text.grid(row=1, column=0, padx=5, pady=5)

    # Add a vertical scrollbar to the Text widget
    scrollbar = ttk.Scrollbar(df_frame, orient="vertical", command=df_text.yview)
    scrollbar.grid(row=1, column=1, sticky='ns')
    df_text.config(yscrollcommand=scrollbar.set)

    # Insert the entire DataFrame into the Text widget
    df_text.insert(tk.END, df.to_string(index=False))

    # Define the background color change function
    def change_background_color(*args):
        # Get the current position of the scrollbar
        current_position = float(df_text.yview()[0])
        # Calculate the color gradient based on the scrollbar position
        color = calculate_color_gradient(current_position, 1.0)
        # Set the background color of the Text widget
        df_text.config(bg=color)

    # Bind the scrollbar's events to the background color change function
    scrollbar.bind("<B1-Motion>", change_background_color)
    scrollbar.bind("<ButtonRelease-1>", change_background_color)
    
    
    
# Function to set lookup step
def set_lookup_step():
    global LOOKUP_STEP
    step = int(step_entry.get())
    LOOKUP_STEP = step

# Function to plot loss metrics and display TensorBoard
import webbrowser
def plot_loss_metrics():
    # Load tensorboard extension
    try:
        subprocess.run(["jupyter", "nbextension", "enable", "--py", "tensorboard"])
        subprocess.run(["jupyter", "serverextension", "enable", "--py", "tensorboard"])
    except Exception as e:
        print("Error loading tensorboard extension:", e)
        return

    # Create log directory if not exists
    log_dir = "logs"
    os.makedirs(log_dir, exist_ok=True)

    # Run tensorboard
    try:
        subprocess.Popen(["tensorboard", "--logdir", log_dir])
    except Exception as e:
        print("Error running tensorboard:", e)
    
    # Open TensorBoard in the default web browser
    webbrowser.open_new_tab("http://localhost:6006/")
    
    
# Function to print metrics
def print_metrics():
    # Printing all the previously calculated metrics
    future_price_str = f"Future price after {LOOKUP_STEP} days is ${future_price:.2f}\n"
    loss_str = f"{LOSS} loss: {loss}\n"
    mae_str = f"Mean Absolute Error: {mean_absolute_error}\n"
    accuracy_str = f"Accuracy: {accuracy_score}\n"
    buy_profit_str = f"Total buy profit: {total_buy_profit}\n"
    sell_profit_str = f"Total sell profit: {total_sell_profit}\n"
    total_profit_str = f"Total profit: {total_profit}\n"
    profit_per_trade_str = f"Profit per trade: {profit_per_trade}\n"

    # Display metrics in a new window
    metrics_window = tk.Toplevel(root)
    metrics_window.title("Metrics")
    metrics_frame = ttk.Frame(metrics_window)
    metrics_frame.pack(padx=10, pady=10)
    metrics_label = ttk.Label(metrics_frame, text="Metrics:")
    metrics_label.grid(row=0, column=0, padx=5, pady=5)
    metrics_text = tk.Text(metrics_frame, height=10, width=50)
    metrics_text.insert(tk.END, future_price_str + loss_str + mae_str + accuracy_str +
                        buy_profit_str + sell_profit_str + total_profit_str + profit_per_trade_str)
    metrics_text.grid(row=1, column=0, padx=5, pady=5)

# Function to plot the prices graph
def plot_prices():
    # Create a new window to display the plot
    plot_window = tk.Toplevel(root)
    plot_window.title("Actual and Predicted Prices")
    plot_frame = ttk.Frame(plot_window)
    plot_frame.pack(padx=10, pady=10)

    # Plot both actual and predicted prices in the new window
    figure = plt.Figure(figsize=(8, 6), dpi=100)
    ax = figure.add_subplot(111)
    ax.plot(final_df.index, final_df[f'true_adjclose_{LOOKUP_STEP}'], label='Actual Price', c='b')
    ax.plot(final_df.index, final_df[f'adjclose_{LOOKUP_STEP}'], label='Predicted Price', c='r')
    ax.set_xlabel('Date')
    ax.set_ylabel('Price')
    ax.set_title('Actual and Predicted Prices')
    ax.legend()
    ax.grid(True)
    canvas = FigureCanvasTkAgg(figure, plot_frame)
    canvas.draw()
    canvas.get_tk_widget().pack(side=tk.TOP, fill=tk.BOTH, expand=True)


# Create a frame for the menu
menu_frame = ttk.LabelFrame(root, text="Menu")
menu_frame.pack(padx=10, pady=10, fill=tk.BOTH, expand=True)

# Menu 1: Stock Symbol
symbol_label = ttk.Label(menu_frame, text="Stock Symbol:")
symbol_label.grid(row=0, column=0, padx=5, pady=5, sticky="e")

symbols = ["AMZN", "NVDA","TSLA","AAPL","GOOGL"]
symbol_combobox = ttk.Combobox(menu_frame, values=symbols, state="readonly")
symbol_combobox.grid(row=0, column=1, padx=5, pady=5)
symbol_combobox.current(0)

load_data_button = ttk.Button(menu_frame, text="Load Data", command=load_data_and_display, style="Custom.TButton")
load_data_button.grid(row=0, column=2, padx=5, pady=5)

# Menu 2: Setting step
step_label = ttk.Label(menu_frame, text="Setting step:")
step_label.grid(row=1, column=0, padx=5, pady=5, sticky="e")

step_entry = ttk.Entry(menu_frame)
step_entry.grid(row=1, column=1, padx=5, pady=5)
step_entry.insert(0, LOOKUP_STEP)

set_step_button = ttk.Button(menu_frame, text="Set Step", command=set_lookup_step, style="Custom.TButton")
set_step_button.grid(row=1, column=2, padx=5, pady=5)

# Menu 3: Plot Loss Metrics
plot_loss_button = ttk.Button(menu_frame, text="Plot Loss Metrics", command=plot_loss_metrics, style="Custom.TButton")
plot_loss_button.grid(row=2, column=0, columnspan=3, padx=5, pady=5, sticky="ew")

# Menu 4: Plot Prices
plot_prices_button = ttk.Button(menu_frame, text="Plot Prices", command=plot_prices, style="Custom.TButton")
plot_prices_button.grid(row=3, column=0, columnspan=3, padx=5, pady=5, sticky="ew")

# Menu 5: Printing metrics
print_metrics_button = ttk.Button(menu_frame, text="Print Metrics", command=print_metrics, style="Custom.TButton")
print_metrics_button.grid(row=4, column=0, columnspan=3, padx=5, pady=5, sticky="ew")

# Styling the buttons
style = ttk.Style()
style.configure("Custom.TButton",
                background="#1f77b4",
                foreground="black",  # Change text color to black
                font=("Helvetica", 10, "bold"),
                padding=10)

# Run the GUI
root.mainloop()
